In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import itertools as IT
import numpy as np

In [8]:
import nltk 
from nltk.tokenize import RegexpTokenizer

file = open('dev-0/in.tsv', 'r', encoding='utf8').readlines()
data = []
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    data.append(new_words)

file = open('dev-1/in.tsv', 'r', encoding='utf8').readlines()
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    data.append(new_words)

file = open('test-A/in.tsv', 'r', encoding='utf8').readlines()
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    data.append(new_words)

In [18]:
file = open('train/in.tsv', 'r', encoding='utf8').readlines()[0:5000]
train_data = []
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    train_data.append(new_words)

file = open('train/expected.tsv', 'r', encoding='utf8').readlines()[0:5000]
train_data_output = []
for line in file:
    train_data_output.append(int(line))

In [19]:
my_dict = {}
for line in data + train_data:
    for word in line:
        if word not in my_dict:
            my_dict[word] = len(my_dict)

In [20]:
DICT_SIZE = len(my_dict)
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(DICT_SIZE, 2)
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred
    
def make_vector(sentence, words):
    vector = torch.zeros(len(words))
    for word in sentence:
        vector[words[word]] += 1
    return vector.view(1, -1)

model = LogisticRegression()

In [24]:
criterion = nn.NLLLoss()
opt = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
epochs = 100
for epoch in range(epochs):
    if epoch % 10 == 0:
        print(str(epoch/epochs * 100) + "%")
    for instance, label in IT.zip_longest(train_data, train_data_output):
        vector = make_vector(instance, my_dict)
        target = torch.LongTensor([label])
        model.zero_grad()
        log_probs = model(vector)
        loss = criterion(log_probs, target)
        loss.backward()
        opt.step() 

0.0%


In [ ]:
file = open('dev-0/in.tsv', 'r', encoding='utf8').readlines()
dev_data = []
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    dev_data.append(new_words)

In [ ]:
f=open('dev-0/out.tsv','w')
for line in dev_data:
        vec = make_vector(line, my_dict)
        log_probs = model(vec)
        y_pred = np.argmax(log_probs[0].detach().numpy())
        f.write(str(int(y_pred)) + '\n')

f.close()

In [ ]:
file = open('dev-1/in.tsv', 'r', encoding='utf8').readlines()
dev_data = []
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    dev_data.append(new_words)

In [ ]:
f=open('dev-1/out.tsv','w')
for line in dev_data:
        vec = make_vector(line, my_dict)
        log_probs = model(vec)
        y_pred = np.argmax(log_probs[0].detach().numpy())
        f.write(str(int(y_pred)) + '\n')

f.close()

In [ ]:
file = open('test-A/in.tsv', 'r', encoding='utf8').readlines()
dev_data = []
for line in file:
    line = line.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(line)
    dev_data.append(new_words)

In [ ]:
f=open('test-A/out.tsv','w')
for line in dev_data:
        vec = make_vector(line, my_dict)
        log_probs = model(vec)
        y_pred = np.argmax(log_probs[0].detach().numpy())
        f.write(str(int(y_pred)) + '\n')

f.close()